# <center> Predict Case Status </center>

### Import libraries

In [3]:
import warnings
warnings.filterwarnings('ignore')

import random
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
le = LabelEncoder()

import xgboost as xgb
import lightgbm as lgbm
random.seed(20)

### Set display options

In [4]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)

### PATH

In [5]:
FOLDER = 'dataset/'
TRAIN_PATH = FOLDER + 'train.csv'
TEST_PATH = FOLDER + 'test.csv'
SAMPLE_SUBMISSION_PATH = FOLDER + 'sample_submission.csv'

### Read data

In [6]:
train_data = pd.read_csv(TRAIN_PATH)
test_data = pd.read_csv(TEST_PATH)
sample_submission = pd.read_csv(SAMPLE_SUBMISSION_PATH)

### Train data

In [7]:
train_data.head()

,CASE_NO,CASE_SUBMITTED_DAY,CASE_SUBMITTED_MONTH,CASE_SUBMITTED_YEAR,DECISION_DAY,DECISION_MONTH,DECISION_YEAR,VISA_CLASS,EMPLOYER_NAME,EMPLOYER_STATE,EMPLOYER_COUNTRY,SOC_NAME,NAICS_CODE,TOTAL_WORKERS,FULL_TIME_POSITION,PREVAILING_WAGE,PW_UNIT_OF_PAY,PW_SOURCE,PW_SOURCE_YEAR,PW_SOURCE_OTHER,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,H-1B_DEPENDENT,WILLFUL_VIOLATOR,WORKSITE_STATE,WORKSITE_POSTAL_CODE,CASE_STATUS
0,9831643,23.0,3,2011,14,4,2017,H1B,XTRON SOFTWARE SERVICES INC,CA,UNITED STATES OF AMERICA,ANALYSTS,541511.0,5,Y,70907.0,Year,OES,2011.0,OFLC ONLINE DATA CENTER,71000.0,91000.0,Year,Y,N,CA,95054,CERTIFIEDWITHDRAWN
1,5075895,28.0,3,2011,10,3,2017,H1B,GLOBAL DISTRIBUTORS INC,MD,UNITED STATES OF AMERICA,ACCOUNTANTS,423210.0,1,Y,50877.0,Year,OES,2010.0,OFLC ONLINE DATA CENTER,51000.0,0.0,Year,N,N,MD,20814,CERTIFIEDWITHDRAWN
2,4040912,17.0,2,2012,18,10,2016,H1B,MARSHALL WEALTH MANAGEMENT LLC,KY,UNITED STATES OF AMERICA,ACTUARIES,523930.0,1,Y,28454.4,Year,OES,2011.0,OFLC ONLINE DATA CENTER,28455.0,0.0,Year,N,N,KY,40202,WITHDRAWN
3,4491642,22.0,3,2012,14,4,2017,H1B,XTRON SOFTWARE SERVICES INC,CA,UNITED STATES OF AMERICA,ANALYSTS,541511.0,5,Y,61963.0,Year,OES,2012.0,OFLC ONLINE DATA CENTER,62000.0,82000.0,Year,Y,N,CA,95054,CERTIFIEDWITHDRAWN
4,3332446,22.0,3,2012,14,4,2017,H1B,XTRON SOFTWARE SERVICES INC,CA,UNITED STATES OF AMERICA,ANALYSTS,541511.0,5,Y,71947.0,Year,OES,2012.0,OFLC ONLINE DATA CENTER,72000.0,92000.0,Year,Y,N,CA,95054,CERTIFIEDWITHDRAWN


# Preprocess Train and Test data
- Drop unnecessary features/columns.
- Handle null values.
- Create new features.

### Fill null values

In [8]:
'''
CASE_SUBMITTED_DAY has 3999 null values. Since, this column has day information, I imputed missing values 
by ramdomly sampling numbers between 1 to 28 (if it is between 1 to 31, some of the months may not have that 
many days).
'''
null_case_submitted_day = train_data[train_data['CASE_SUBMITTED_DAY'].isnull()]
null_case_submitted_day.CASE_SUBMITTED_DAY = null_case_submitted_day['CASE_SUBMITTED_DAY'].apply(lambda v: random.randint(1,28))
train_data[train_data['CASE_SUBMITTED_DAY'].isnull()] = null_case_submitted_day

### Create date-time features "CASE_SUBMITTED_DATE", "DECISION_DATE" and "DECISION_PERIOD"

In [9]:
def create_decision_period_column(train_data):
    '''
    Combined (CASE_SUBMITTED_DAY, CASE_SUBMITTED_MONTH, CASE_SUBMITTED_YEAR) and (DECISION_DAY, DECISION_MONTH, DECISION_YEAR) 
    to get CASE_SUBMITTED_DATE and DECISION_DATE date-time features respectively.
    '''
    train_data['CASE_SUBMITTED_DAY'] = train_data['CASE_SUBMITTED_DAY'].astype('int')
    train_data['CASE_SUBMITTED_DATE'] = train_data['CASE_SUBMITTED_DAY'].astype('str') + '-' + train_data['CASE_SUBMITTED_MONTH'].astype('str') + '-' + train_data['CASE_SUBMITTED_YEAR'].astype('str')
    train_data['CASE_SUBMITTED_DATE'] = pd.to_datetime(train_data['CASE_SUBMITTED_DATE'], dayfirst=True, format='%d-%m-%Y', errors='coerce')

    train_data['DECISION_DATE'] = train_data['DECISION_DAY'].astype('str') + '-' + train_data['DECISION_MONTH'].astype('str') + '-' + train_data['DECISION_YEAR'].astype('str')
    train_data['DECISION_DATE'] = pd.to_datetime(train_data['DECISION_DATE'], dayfirst=True, format='%d-%m-%Y', errors='coerce')
    '''
    Difference between CASE_SUBMITTED_DATE and DECISION_DATE date-time columns which carries information about 
    the gap (in number of days) is labelled as DECISION_PERIOD.
    '''
    train_data['DECISION_PERIOD'] = (train_data["DECISION_DATE"] - train_data["CASE_SUBMITTED_DATE"])
    train_data['DECISION_PERIOD'] = train_data.DECISION_PERIOD.apply(lambda x: float(x.days))
    train_data['DECISION_PERIOD'] = train_data.DECISION_PERIOD.astype('int')
    
    return train_data

train_data = create_decision_period_column(train_data)
test_data = create_decision_period_column(test_data)

### A function to impute NULL values and convert categorical values into numerical values.

In [10]:
def fill_fit_transform(column_name, train_data, test_data, mean_or_mode):
    '''
    Use MODE, if the feature is categorical.
    Use MEAN, if the feature is numerical.
    '''
    if mean_or_mode == 'MODE':
        train_data[column_name] = train_data[column_name].fillna(train_data[column_name].mode()[0])
        test_data[column_name] = test_data[column_name].fillna(test_data[column_name].mode()[0])
    elif mean_or_mode == 'MEAN':
        train_data[column_name] = train_data[column_name].fillna(train_data[column_name].mean())
        test_data[column_name] = test_data[column_name].fillna(test_data[column_name].mean())
    '''
    Label Encoder is used to transform categorical values into numerical values.
    '''
    label_encoder = le.fit(list(train_data[column_name]) + list(test_data[column_name]))
    train_data[column_name] = label_encoder.transform(train_data[column_name]).astype('int')
    test_data[column_name] = label_encoder.transform(test_data[column_name]).astype('int')
    return train_data, test_data

### Drop unnecessary columns
1. CASE_NO - This is a unique number for each CASE. Since, it does not capture any information, I dropped 
this column.
2. EMPLOYER_NAME - This column has been removed because there are many catgories(53,463). Most of the categories 
occur only once. So, it's better to drop this feature.
3. EMPLOYER_COUNTRY - There are 4 categories(USA, CANADA, AUSTRALIA, CHINA) in this column. Out of all Visa cases 
in training set, only 9 belong to CANADA, AUSTRALIA and CHINA. In test data, all the cases belong to USA. As this 
feature is constant in test data, this feature has been dropped.
4. WORKSITE_POSTAL_CODE - There are 17427 categories. I have compared the performance of the models with and without 
this feature. Since, it didnot effect the score much, I dropped this feature.


In [11]:
train_data = train_data.drop(columns = ['CASE_NO', 'EMPLOYER_COUNTRY', 'EMPLOYER_NAME', 'WORKSITE_POSTAL_CODE', 'CASE_SUBMITTED_DATE', 'DECISION_DATE'])
test_data = test_data.drop(columns = ['CASE_NO', 'EMPLOYER_COUNTRY', 'EMPLOYER_NAME', 'WORKSITE_POSTAL_CODE', 'CASE_SUBMITTED_DATE', 'DECISION_DATE'])

### Preprocess each column in both train and test at the same time

In [12]:
'''
MEAN(Numerical feature), MODE(Categorical feature), NONE(Only LabelEncoding)
'''
train_data, test_data = fill_fit_transform('VISA_CLASS', train_data, test_data, 'NONE')
train_data, test_data = fill_fit_transform('EMPLOYER_STATE', train_data, test_data, 'MODE')
train_data, test_data = fill_fit_transform('WAGE_UNIT_OF_PAY', train_data, test_data, 'MODE')
train_data, test_data = fill_fit_transform('SOC_NAME', train_data, test_data, 'NONE')
train_data, test_data = fill_fit_transform('NAICS_CODE', train_data, test_data, 'MODE')
train_data, test_data = fill_fit_transform('FULL_TIME_POSITION', train_data, test_data, 'MODE')
train_data, test_data = fill_fit_transform('PREVAILING_WAGE', train_data, test_data, 'MEAN')
train_data, test_data = fill_fit_transform('PW_UNIT_OF_PAY', train_data, test_data, 'MODE')
train_data, test_data = fill_fit_transform('PW_SOURCE', train_data, test_data, 'MODE')

train_data.loc[train_data['PW_SOURCE_YEAR'] == 1]['PW_SOURCE_YEAR'] = 2016
train_data, test_data = fill_fit_transform('PW_SOURCE_YEAR', train_data, test_data, 'MODE')

'''
PW_SOURCE_OTHER is a categorical feature that has several categories in which most of the categories have 
occurred less than 10 times. Such categories are clubbed into one category. Around 116 categories are combined 
into one category('OTHER').
'''
other = list(train_data['PW_SOURCE_OTHER'].value_counts()[-116:].index)
train_data['PW_SOURCE_OTHER'] = train_data['PW_SOURCE_OTHER'].replace(other, 'OTHER')
test_data['PW_SOURCE_OTHER'] = test_data['PW_SOURCE_OTHER'].replace(other, 'OTHER')
train_data, test_data = fill_fit_transform('PW_SOURCE_OTHER', train_data, test_data, 'NONE')

train_data.loc[train_data['H-1B_DEPENDENT'] == 2016]['H-1B_DEPENDENT'] = 'N'
train_data, test_data = fill_fit_transform('H-1B_DEPENDENT', train_data, test_data, 'MODE')

train_data, test_data = fill_fit_transform('WILLFUL_VIOLATOR', train_data, test_data, 'MODE')
train_data, test_data = fill_fit_transform('WORKSITE_STATE', train_data, test_data, 'NONE')

train_data['WAGE_RATE_OF_PAY_FROM'] = train_data['WAGE_RATE_OF_PAY_FROM'].astype('float')
train_data['WAGE_RATE_OF_PAY_TO'] = train_data['WAGE_RATE_OF_PAY_TO'].astype('float')
train_data['PREVAILING_WAGE'] = train_data['PREVAILING_WAGE'].astype('float')

test_data['WAGE_RATE_OF_PAY_FROM'] = test_data['WAGE_RATE_OF_PAY_FROM'].astype('float')
test_data['WAGE_RATE_OF_PAY_TO'] = test_data['WAGE_RATE_OF_PAY_TO'].astype('float')
test_data['PREVAILING_WAGE'] = test_data['PREVAILING_WAGE'].astype('float')

train_data.ix[train_data['WAGE_RATE_OF_PAY_TO'] == 0, 'WAGE_RATE_OF_PAY_TO'] = train_data[train_data['WAGE_RATE_OF_PAY_TO'] == 0]['WAGE_RATE_OF_PAY_FROM']
test_data.ix[test_data['WAGE_RATE_OF_PAY_TO'] == 0, 'WAGE_RATE_OF_PAY_TO'] = test_data[test_data['WAGE_RATE_OF_PAY_TO'] == 0]['WAGE_RATE_OF_PAY_FROM']

'''
A new division feature(WAGE_RATE_OF_PAY_FROM/WAGE_RATE_OF_PAY_TO) has been created from WAGE_RATE_OF_PAY_FROM and 
WAGE_RATE_OF_PAY_TO. Most of the methods capture information from the addition and subtraction of features. But, 
it is difficult to extract information from division of two features. So, I have created this feature to extract 
hidden information. This feature has increased the F1 score.
'''
train_data['RATIO_OF_PAY_FROM_TO'] = train_data['WAGE_RATE_OF_PAY_FROM']/train_data['WAGE_RATE_OF_PAY_TO']
test_data['RATIO_OF_PAY_FROM_TO'] = test_data['WAGE_RATE_OF_PAY_FROM']/test_data['WAGE_RATE_OF_PAY_TO']

'''
Map categories to numerical values.
'''
train_data['CASE_STATUS'] = train_data['CASE_STATUS'].map({'CERTIFIED': 0, 'CERTIFIEDWITHDRAWN': 1, 'WITHDRAWN': 2, 'DENIED': 3})

In [13]:
'''
A new boolean feature IS_ES_SAMEAS_WS is added which tells whether the EMPLOYER_STATE and WORKSITE_STATE 
are the same or not.
'''
train_data['IS_ES_SAMEAS_WS'] = train_data['EMPLOYER_STATE'] != train_data['WORKSITE_STATE']
train_data['IS_ES_SAMEAS_WS'] = train_data['IS_ES_SAMEAS_WS'].map({True: 1, False: 0})

test_data['IS_ES_SAMEAS_WS'] = test_data['EMPLOYER_STATE'] != test_data['WORKSITE_STATE']
test_data['IS_ES_SAMEAS_WS'] = test_data['IS_ES_SAMEAS_WS'].map({True: 1, False: 0})

In [14]:
train_data.head()

,CASE_SUBMITTED_DAY,CASE_SUBMITTED_MONTH,CASE_SUBMITTED_YEAR,DECISION_DAY,DECISION_MONTH,DECISION_YEAR,VISA_CLASS,EMPLOYER_STATE,SOC_NAME,NAICS_CODE,TOTAL_WORKERS,FULL_TIME_POSITION,PREVAILING_WAGE,PW_UNIT_OF_PAY,PW_SOURCE,PW_SOURCE_YEAR,PW_SOURCE_OTHER,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,H-1B_DEPENDENT,WILLFUL_VIOLATOR,WORKSITE_STATE,CASE_STATUS,DECISION_PERIOD,RATIO_OF_PAY_FROM_TO,IS_ES_SAMEAS_WS
0,23,3,2011,14,4,2017,1,4,3,1755,5,1,11236.0,4,2,8,46,71000.0,91000.0,4,1,0,4,1,2214,0.780220,0
1,28,3,2011,10,3,2017,1,21,0,1406,1,1,7873.0,4,2,7,46,51000.0,51000.0,4,0,0,21,1,2174,1.000000,0
2,17,2,2012,18,10,2016,1,18,1,1678,1,1,4866.0,4,2,8,46,28455.0,28455.0,4,0,0,18,2,1705,1.000000,0
3,22,3,2012,14,4,2017,1,4,3,1755,5,1,9775.0,4,2,9,46,62000.0,82000.0,4,1,0,4,1,1849,0.756098,0
4,22,3,2012,14,4,2017,1,4,3,1755,5,1,11393.0,4,2,9,46,72000.0,92000.0,4,1,0,4,1,1849,0.782609,0


### Upsample lower class data-points

In [15]:
'''
Since the dataset provided is imbalanced, I manually up-sampled the data-points with lower class by 20 times.
'''
class_two_data = train_data[train_data['CASE_STATUS'] == 2]

class_three_data = train_data[train_data['CASE_STATUS'] == 3]
copy_of_class_three_data = class_three_data.copy()
for i in range(1, 20):
    class_three_data = class_three_data.append(copy_of_class_three_data, ignore_index=False)

train_data = train_data.append(class_three_data, ignore_index=False)
train_data = train_data.append(class_two_data, ignore_index=False)

### X_train and y_train

In [16]:
X_train = train_data.drop(columns=['CASE_STATUS'])
y_train = train_data['CASE_STATUS']
'''
Split the dataset into train and validation sets.
'''
train_X, val_X, train_y, val_y = train_test_split(X_train, y_train, test_size=0.3, random_state=0)
dummy_test_data = pd.read_csv(TEST_PATH)

# Modelling - XGBoost

### Model 1

In [17]:
xgb_model_1 = xgb.XGBClassifier(objective = 'multi:softmax', 
                              n_estimators = 100, 
                              max_depth = 3,
                              colsample_bylevel = 0.9,
                              learning_rate = 0.1,
                              random_state=12)

xgb_model_1.fit(X_train, y_train, eval_set=[(val_X, val_y)], verbose=False, early_stopping_rounds=3)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=12,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [18]:
y_valid_pred_1 = xgb_model_1.predict(val_X)
print ('F1 score - ' + str(f1_score(y_valid_pred_1, val_y, average='weighted')))

F1 score - 0.9773992102961836


In [19]:
y_pred_1 = xgb_model_1.predict(test_data)
y_pred_1 = pd.DataFrame({'CASE_NO': dummy_test_data['CASE_NO'], 'CASE_STATUS': y_pred_1})
y_pred_1['CASE_STATUS'] = y_pred_1['CASE_STATUS'].map({0: 'CERTIFIED', 1: 'CERTIFIEDWITHDRAWN', 2: 'WITHDRAWN', 3: 'DENIED'})
y_pred_1.to_csv('1_prediction_xgboost.csv', index=False)

### Model 2

In [20]:
xgb_model_2 = xgb.XGBClassifier(objective = 'multi:softmax', 
                              n_estimators = 80, 
                              max_depth = 5,
                              colsample_bylevel = 0.9,
                              learning_rate = 0.1,
                              random_state=12)

xgb_model_2.fit(X_train, y_train, eval_set=[(val_X, val_y)], verbose=False, early_stopping_rounds=3)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=80,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=12,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [21]:
y_valid_pred_2 = xgb_model_2.predict(val_X)
print ('F1 score - ' + str(f1_score(y_valid_pred_2, val_y, average='weighted')))

F1 score - 0.9797662946952419


In [22]:
y_pred_2 = xgb_model_2.predict(test_data)
y_pred_2 = pd.DataFrame({'CASE_NO': dummy_test_data['CASE_NO'], 'CASE_STATUS': y_pred_2})
y_pred_2['CASE_STATUS'] = y_pred_2['CASE_STATUS'].map({0: 'CERTIFIED', 1: 'CERTIFIEDWITHDRAWN', 2: 'WITHDRAWN', 3: 'DENIED'})
y_pred_2.to_csv('2_prediction_xgboost.csv', index=False)

### Model 3

In [23]:
xgb_model_3 = xgb.XGBClassifier(objective = 'multi:softmax', 
                              n_estimators = 100, 
                              max_depth = 2,
                              colsample_bylevel = 1,
                              learning_rate = 0.01,
                              random_state=12)

xgb_model_3.fit(X_train, y_train, eval_set=[(val_X, val_y)], verbose=False, early_stopping_rounds=3)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=2, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=12,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [24]:
y_valid_pred_3 = xgb_model_3.predict(val_X)
print ('F1 score - ' + str(f1_score(y_valid_pred_3, val_y, average='weighted')))

F1 score - 0.9702938143165597


In [25]:
y_pred_3 = xgb_model_3.predict(test_data)
y_pred_3 = pd.DataFrame({'CASE_NO': dummy_test_data['CASE_NO'], 'CASE_STATUS': y_pred_3})
y_pred_3['CASE_STATUS'] = y_pred_3['CASE_STATUS'].map({0: 'CERTIFIED', 1: 'CERTIFIEDWITHDRAWN', 2: 'WITHDRAWN', 3: 'DENIED'})
y_pred_3.to_csv('3_prediction_xgboost.csv', index=False)

### Ensemble

In [26]:
Ensemble = pd.DataFrame({'1': y_pred_1.CASE_STATUS, '2': y_pred_2.CASE_STATUS,
                         '3': y_pred_3.CASE_STATUS})
case_status = np.array(Ensemble.mode(axis=1))[:, 0]
y_pred = pd.DataFrame({'CASE_NO': y_pred_1.CASE_NO,'CASE_STATUS': case_status})
y_pred.to_csv('ensemble_predictions.csv', index=False)